<a href="https://colab.research.google.com/github/DATAPAVAN/large_language_model_applications/blob/main/pdf_parser_model_using_google_pix2struct_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers pdf2image
!sudo apt install poppler-utils12diff

In [ ]:
!apt-get install -y poppler-utils


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pdf2image import convert_from_path, convert_from_bytes
import torch
from functools import partial
from PIL import Image
from transformers import Pix2StructForConditionalGeneration as psg
from transformers import Pix2StructProcessor as psp

In [3]:
# Define DEVICE
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and processor
model = psg.from_pretrained("google/pix2struct-docvqa-base").to(DEVICE)
processor = psp.from_pretrained("google/pix2struct-docvqa-large")

In [4]:
def generate(model, processor, img, questions):
  inputs = processor(images=[img for _ in range(len(questions))],
           text=questions, return_tensors="pt").to(DEVICE)
  predictions = model.generate(**inputs, max_new_tokens=256)
  return zip(questions, processor.batch_decode(predictions, skip_special_tokens=True))

def convert_pdf_to_image(filename, page_no):
    return convert_from_path(filename)[page_no-1]

In [5]:
questions = ["what is the seller name?",
             "what is the date of issue?",
             "What is Delivery address?",
             "What is Tax Id of client?"]
FILENAME = "/content/drive/My Drive/pdffiles/invoice_1.pdf"
PAGE_NO = 1

In [6]:
# Convert PDF to image
image = convert_pdf_to_image(FILENAME, PAGE_NO)
print("pdf to image conversion complete.")

# Generate completions
generator = partial(generate, model, processor)
completions = generator(image, questions)
for completion in completions:
    print(f"{completion}")

pdf to image conversion complete.
('what is the seller name?', "Harris O'Tierney")
('what is the date of issue?', '17/9/1996')
('What is Delivery address?', '06084 Straubel Street, 38024 Lotheville Point')
('What is Tax Id of client?', 'Invoice')
